In [24]:
from __future__ import division

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

import math
import numpy as np
import cv2
import os
import glob

import random

In [25]:
#Constants - resized image 

reshapedImageHeight = 80
reshapedImageWidth = 25
sizeOfContour = 150

In [26]:
#Globals

means= []
variances = []

In [27]:
def getMean2DVector(vector):
    meanX = 0.0
    meanY = 0.0

    for v in vector:
        meanX = meanX + v[0]
        meanY = meanY + v[1]
    return [meanX / len(vector),meanY / len(vector)]

In [28]:
def getVariance2DVector(vector, meanX, meanY):
    varX = 0.0
    varY = 0.0
    for v in vector:
        varX = varX + (meanX - v[0])**2
        varY = varY + (meanY - v[1])**2
    return [math.sqrt(varX / (len(vector) - 1)), math.sqrt(varY / (len(vector) - 1))]

In [29]:
def isEdgePoint(i,j,image):
    
    if(image[i][j]==255 and (image[i-1][j-1]==0 or image[i-1][j]==0 or image[i-1][j+1]==0 or image[i][j-1]==0 or image[i][j+1]==0 or image[i+1][j-1]==0 or image[i+1][j]==0 or image[i+1][j+1]==0)):
        return True
    return False

In [30]:
#find points on edge
def getPointsFromContours(image):
    
    contour=[]
    
    index=0
    
    for i in range (1, reshapedImageHeight-1):
        for j in range (1, reshapedImageWidth-1):
            if isEdgePoint(i,j,image) ==True:
                contour.append([i,j])
    return contour
    

In [31]:
# select same number of points in each contour
def densityFix(contours):
    
    difference = len(contours) - sizeOfContour
    
    multiple = len(contours)/ difference
    
    addedContour = []
    
    index = 1 
    multipleIndex = 1
    
    for c in contours:
        if(index >= multiple * multipleIndex):
            multipleIndex =  multipleIndex + 1
        else:
            addedContour.append(c)
        index = index + 1
            
    if(len(addedContour) == sizeOfContour + 1):
        addedContour.pop()
        
    return addedContour

In [32]:
def extractContours():
    
    imagePath = "PedCut2013_SegmentationDataset/data/completeData/left_groundTruth/*.png"
    
    contours = []
    
    i=0
    
    for filename in glob.glob(imagePath):
        
        image= cv2.imread(filename, 0)
        image = cv2.resize(image, (reshapedImageWidth,reshapedImageHeight),interpolation=cv2.INTER_NEAREST)
        
        contour = getPointsFromContours(image)
        
        contour = densityFix(contour)
        
        #for c in contour:
        #    contours.append(c)
        
        contours.append(contour)
        
    contours = np.array(contours)
    return contours

In [33]:
def getCovarianceFromMatrix(matrix):
    
    print(matrix.shape)
    covarianceMatrix = np.ndarray((matrix.shape[1], matrix.shape[1]))
    
    for i in range(matrix.shape[1]):
        for j in range(matrix.shape[1]):
        
            rez = 0
            for z in range (matrix.shape[0]):
                rez = rez + matrix[z][i][0] * matrix[z][j][0] + matrix[z][i][1] * matrix[z][j][1]
            covarianceMatrix[i][j] = rez
        
    return covarianceMatrix

In [34]:
def adjustContour(point):
    
    correctPoint = [int(point[0]), int(point[1])]
    
    if(point[0] >= reshapedImageHeight ):
        correctPoint[0] = reshapedImageHeight - 1
    if(point[1] >= reshapedImageWidth ):
        correctPoint[1] = reshapedImageWidth - 1
        
    return correctPoint

In [35]:
def drawImageFromContour(contour):
    
    contourImage = np.zeros((reshapedImageHeight,reshapedImageWidth))
    for i in range(contour.shape[0]):
    
        adjustedContour = adjustContour(contour[i])
        
        contourImage[adjustedContour[0]][adjustedContour[1]] = 255
    
    cv2.imshow('Shape of pedestrian', contourImage)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    

In [36]:
def drawAverageOfAllImages(contours):
    
    contourImage = np.zeros((reshapedImageHeight,reshapedImageWidth))
    
    sumOfContours = np.zeros((contours.shape[1],2))
    for j in range(contours.shape[0]):

        sumOfContours += contours[j]

    sumOfContours = sumOfContours / contours.shape[0]
    
    drawImageFromContour(sumOfContours)
    

In [37]:
inputData = extractContours()

#complexData = convertToImaginary(inputData)

#print(inputData)

In [38]:
def preprocess(inputV):

    reshapedV = np.reshape(inputV,(inputV.shape[0] * inputV.shape[1], inputV.shape[2]))
    means = getMean2DVector(reshapedV)
    variances = getVariance2DVector(reshapedV, means[0], means[1])
   
    outputV = []
    
    for v in reshapedV:
        x = (v[0] - means[0]) / variances[0]
        y = (v[1] - means[1]) / variances[1]
        
        outputV.append([x,y])
    return np.reshape(np.array(outputV), (inputV.shape[0],inputV.shape[1],inputV.shape[2]))

In [39]:
def pca(inputData, nrComponents):
    
    size = inputData.shape
    print(size)
    
    #covariance = np.dot(inputData.T,inputData)
    
    covariance = getCovarianceFromMatrix(inputData)
    covariance = covariance / size[1]
    
    print(covariance)
    
    eigenvalues, eigenvectors = np.linalg.eig(covariance)
    
    #print(eigenvalues)
    
    selectedEigenvectors = np.ndarray((nrComponents,eigenvectors.shape[1]))

    sortedEigen = np.argsort( -1 * eigenvalues)
    
    
    for i in range (nrComponents):
        
        index = sortedEigen[i]
        selectedEigenvectors[i] = eigenvectors.T[index]
        
    XPCA = np.dot(inputData,np.transpose(selectedEigenvectors))
    
    #XTilda= np.dot(XPCA, selectedEigenvectors)
    
    return XPCA
  

In [40]:
# vector of ordered indices
def constructVectorOrder(vector, size):
    
    orderedVector = np.ndarray((size))
    for i in range (size):
        orderedVector[i] = np.argwhere(vector==i)[0]
        
    return orderedVector

In [41]:
def mergeArrays(x, y):
    
    size = x.shape
    
    result = np.ndarray((size[0], size[1],2))
    
    for i in range (size[0]):
        for j in range(size[1]):
            result[i][j][0] = x[i][j]
            result[i][j][1] = y[i][j]
    return result

In [42]:
# extract PCA for each axis (X and Y), choose PCA value with 
def doublePCA(inputData, nrComponents):
    
    size = inputData.shape
    transposedInput = np.transpose(inputData)
    
    x =transposedInput[0]
    y =transposedInput[1]
    
    covarianceX = np.dot(x,x.T)
    covarianceY = np.dot(y,y.T)
    
    covarianceX = covarianceX / size[1]
    covarianceY = covarianceY / size[1]
    
    
    eigenvaluesX, eigenvectorsX = np.linalg.eig(covarianceX)
    eigenvaluesY, eigenvectorsY = np.linalg.eig(covarianceY)
    
    #print(eigenvalues)
    
    selectedEigenvectorsX = np.ndarray((nrComponents,eigenvectorsX.shape[1]))
    selectedEigenvectorsY = np.ndarray((nrComponents,eigenvectorsY.shape[1]))
    
    sortedEigenX = np.argsort(eigenvaluesX)
    sortedEigenY = np.argsort(eigenvaluesY)
    
    pcaOrderOfX = constructVectorOrder(sortedEigenX, size[1])
    pcaOrderOfY = constructVectorOrder(sortedEigenY, size[1])
    
    finalSum = pcaOrderOfX + pcaOrderOfY
    
    sortedFinalSum = np.argsort(finalSum)
    
    selectedTopFeatures = constructVectorOrder(sortedFinalSum, nrComponents)
    
    selectedEigenvectorsX = np.ndarray((nrComponents,eigenvectorsX.shape[1]))
    selectedEigenvectorsY = np.ndarray((nrComponents,eigenvectorsY.shape[1]))
    
    for i in range (nrComponents):
        
        index = int(selectedTopFeatures[i])
        selectedEigenvectorsX[i] = eigenvectorsX.T[index]
        selectedEigenvectorsY[i] = eigenvectorsY.T[index]
        
    XPCA = np.dot(np.transpose(x), np.transpose(selectedEigenvectorsX))
    YPCA = np.dot(np.transpose(y), np.transpose(selectedEigenvectorsY))
    
    XTilda = np.dot(XPCA, selectedEigenvectorsX)
    YTilda = np.dot(YPCA, selectedEigenvectorsY)
    
    return mergeArrays(XTilda,YTilda)

In [43]:
points = np.round(doublePCA(inputData,110))

#draw 20 images to view extracted features
for i in range(20) :
    drawImageFromContour(points[i])
    
drawAverageOfAllImages(points)

#print(inputData)